Selecting files from the alignment and convert to counts

In [6]:
%%bash 
  # featurecounts from bioconda subread
# Define paths
ALIGNED_DIR="data/alignment_STAR_subset/"  
OUTPUT_DIR="data/gene_counts_DESeq"  # Define output directory
GTF_FILE="/human_genome/cleaned_Homo_sapiens.GRCh38.113.gtf"  # GTF annotation file

# Create output directory 
mkdir -p "$OUTPUT_DIR"

# Loop through BAM files
for file in ${ALIGNED_DIR}/*sortedByCoord.out.bam; do
    # Extract sample name
    SAMPLE=$(basename "$file" _Aligned.sortedByCoord.out.bam)

    # Run featureCounts
    featureCounts -T 8 \
        -a "human_genome/Homo_sapiens.GRCh38.113.gtf" \
        -o "${OUTPUT_DIR}/${SAMPLE}.counts.txt" \
        -g gene_id \
        -t exon \
        -s 0 \
        -p \
        "$file"
done



/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory

        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.8

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           ERR950158_trimmed_Aligned.sortedByCoord.out.bam  ||
||                   

merge all files into a count matrix

In [12]:
import os
import pandas as pd

# Define directory
count_dir = "data/gene_counts_DESeq/"

# List all count files
count_files = [f for f in os.listdir(count_dir) if f.endswith("trimmed.counts.txt")]

# Read and merge count files
dfs = []
for file in count_files:
    df = pd.read_csv(os.path.join(count_dir, file), sep="\t", skiprows=1, usecols=[0, 6])  # Geneid & counts
    sample_name = file.replace("trimmed.counts.txt", "")
    df.rename(columns={df.columns[1]: sample_name}, inplace=True)
    dfs.append(df)

# Merge all dataframes
merged_counts = dfs[0]
for df in dfs[1:]:
    merged_counts = merged_counts.merge(df, on="Geneid", how="outer")

# Save merged file
merged_counts.to_csv("merged_counts_matrix.csv", index=False)




Merged count matrix saved as 'merged_counts_matrix.csv'.


Run DESeq2

In [14]:
r_deseq_script = """
library(DESeq2)

# Load count matrix
count_matrix <- read.csv("data/merged_counts_matrix.csv", row.names = 1)

sample_info <- data.frame(
    row.names = colnames(count_matrix),
    condition = c("control", "control", "treated", "treated")
)

# Convert to factors
sample_info$condition <- factor(sample_info$condition, levels = c("control", "treated"))

# Create DESeq2 data
dds <- DESeqDataSetFromMatrix(countData = as.matrix(count_matrix), colData = sample_info, design = ~ condition)

# Run DESeq2
dds <- DESeq(dds)

# Extract results
res <- results(dds, contrast = c("condition", "treated", "control"))

# Save results
write.csv(res, "STAR.DESeq2_results.csv")
"""

# Run DESeq2 inside Python
robjects.r(r_deseq_script)



R[write to console]: Error in data.frame(row.names = colnames(count_matrix), condition = c("control",  : 
  row names supplied are of the wrong length



RRuntimeError: Error in data.frame(row.names = colnames(count_matrix), condition = c("control",  : 
  row names supplied are of the wrong length
